# RSI Swing Trading Strategy- Back Testing

#### This project implements and backtests an RSI-based trading strategy using historical stock data. The strategy involves multiple buy conditions based on RSI values, with profit and averaging criteria. It tracks and reports trading results, including holding periods

#### Here is the rule

1. We will Purchase the stock only if RSI is below 35 and above 30
2. It is a swing trading strategy hence we will sell the stock after we achieve the profit target of 6.28%
3. If the stock is falling then we use the below rule to average it
4. From the 1st buy if the stock has fallen below -3.14% AND stock RSI is below 30 but above 25 then we will initate the 2nd buy
5. We will initiate the 3rd buy if the stock has fallen more than -3.14% from the 2nd buy (not the average) AND stock RSI is below 25 but above 20
6. Fourth buy will be initiated if the stock has fallen more than -3.14% from the 3rd buy (not the average) AND stock RSI is below 20 but above 15
7. Always remember that -3.14% is from the last buy and not the average
8. Whenever we achieve the profit target of 6.28% from the average buy price we will sell all the shares

#### Why 6.28% and -3.14% ?

3.14 is the value of Pi and I believe its a very strong number hence I am using -3.14 for averaging and 2x of Pi (6.28) for profit booking

We will need to the below libraries
1. pandas
2. yfinance
3. pandas_ta
4. math

##### Pandas
The pandas library is used for data manipulation and analysis. It provides powerful data structures like DataFrames, which make it easy to handle, analyze, and visualize large datasets

#### yfinance
The yfinance library is used in Python for fetching historical stock market data.

yfinance allows you to download historical market data directly from Yahoo Finance. This data includes stock prices, trading volume, and other financial metrics like RSI

yfinance returns data in the form of a Pandas DataFrame, which is a versatile data structure that allows easy manipulation and analysis of the data. This is particularly useful for backtesting trading strategies and performing data analysis.

Unlike some other financial data providers, yfinance does not require an API key. This makes it easier and more accessible for users who want to quickly access stock data without additional setup.

You need to install yfinance in your enviornment 1st use below if you are using Jupyter Notebook

pip install yfinance pandas_ta

#### pandas_ta
The pandas_ta library is used for technical analysis in financial markets. It provides a wide range of built-in functions to calculate technical indicators and trading signals, such as:

RSI (Relative Strength Index),
Moving Averages (SMA, EMA),
MACD (Moving Average Convergence Divergence) and
Bollinger Bands

This library integrates with Pandas DataFrames, making it easy to apply technical analysis directly to financial data for backtesting and strategy development.

#### math
The math library in Python provides mathematical functions and constants. In short, it helps with precise mathematical computations needed for financial calculations.

#### Coding

In [2]:
#importing the libraries will we will be using
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import math
from datetime import datetime

#### We will 1st built a function to fetch the stock data from yfinance

In [4]:
# Function to fetch stock data and calculate RSI
def get_stock_data(ticker, start_date, end_date): 
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data['RSI'] = ta.rsi(stock_data['Close'], length=14)
    stock_data.dropna(inplace=True)  # Drop rows with NaN values
    return stock_data

# Parameters:
# ticker: The stock symbol (e.g., 'NS.RELIANCE' for Reliance).
# start_date: The start date for fetching historical data (e.g., '2023-01-01').
# end_date: The end date for fetching historical data (e.g., '2024-01-01').

# stock_data = yf.download(ticker, start=start_date, end=end_date)
# Uses the yfinance library (yf) to download historical stock price data for the specified ticker within the given date range.
# stock_data is a Pandas DataFrame containing columns such as Open, High, Low, Close, and Volume.

# stock_data['RSI'] = ta.rsi(stock_data['Close'], length=14)
# Uses the pandas_ta library (ta) to calculate the RSI based on the 'Close' prices.
# length=14 specifies a 14-day period for the RSI calculation.
# Adds a new column RSI to the DataFrame with the calculated RSI values.

####  The calculate_quantity function determines how many shares of a stock you can buy based on a specified investment amount and the current stock price

In [5]:
# Function to calculate quantity to buy based on investment amount and stock price
def calculate_quantity(investment_amount, price):
    qty = investment_amount / price
    return math.ceil(qty)  # Round up to the nearest whole number


# Uses math.ceil() to round up the calculated quantity to the nearest whole number. This ensures you buy a whole number of 
# shares, as partial shares are not typically purchasable.
# Continuing the example, math.ceil(10.6667) would round up to 11 shares.

#### The function calculate_average_buy_price computes the average buy price of a stock based on the positions held.

In [6]:
# Function to calculate average buy price based on positions
def calculate_average_buy_price(positions):
    total_cost = sum([p['price'] * p['qty'] for p in positions]) #calculates the total_cost by multiplying the price and quantity for each position
    total_qty = sum([p['qty'] for p in positions]) #sums the total quantity (total_qty) of shares bought.
    return total_cost / total_qty #the average buy price is the total cost divided by the total quantity of shares

#### The trading_logic function implements a trading strategy based on RSI (Relative Strength Index) and price movement.

In [10]:
# Trading logic with hold days calculation and multi-buy logic
def trading_logic(stock_data):
    positions = []  # Track each buy position with its buy price and quantity
    initial_buy = False   # flag to indicate if the initial buy has been made
    total_profit_inr = 0  #To accumulate total profit in INR.
    first_buy_date = None #To record the date of the first buy
    results = [] #A list to store trading actions and their details

    # RSI and trading thresholds are initialized, such as RSI ranges for different buy stages, profit target, and stop-loss.

    first_buy_rsi_min = 30
    first_buy_rsi_max = 35
    second_buy_rsi_min = 25
    second_buy_rsi_max = 30
    third_buy_rsi_min = 20
    third_buy_rsi_max = 25
    fourth_buy_rsi_min = 15
    fourth_buy_rsi_max = 20
    profit_target = 6.28 / 100
    stop_loss = -3.14 / 100
    investment_per_buy = 16000  # Each buy is INR 16,000

    max_hold_days = 0
    max_hold_period = None

    # Iterate through each row of the stock data (price and RSI)
    
    for index, row in stock_data.iterrows():
        close_price = row['Close']
        rsi = row['RSI']
        current_date = row.name  # Date from stock data

        # Buy condition: First buy when RSI is between 30 and 35
        if not initial_buy and first_buy_rsi_min < rsi < first_buy_rsi_max: #If initial_buy is False and RSI is within the range for the first buy, calculate the quantity to buy and update positions
            qty = calculate_quantity(investment_per_buy, close_price)
            amount_invested = qty * close_price
            positions.append({'price': close_price, 'qty': qty})
            average_buy_price = calculate_average_buy_price(positions)
            first_buy_date = current_date
            initial_buy = True #Record details of the buy action and set initial_buy to True
            results.append({
                'Date': current_date,
                'Stock': ticker,
                'Action': 'Buy',
                'Price': close_price,
                'RSI': rsi,
                'Quantity': qty,
                'Avg Buy Price': average_buy_price,
                'Amount Invested': amount_invested,
                'Hold Days': 0,  # No hold days yet
                'Profit INR': 0
            })

        # Check for sell condition
        # If initial_buy is True, check if the current price meets or exceeds the target price (buy price plus profit target).
        # Calculate profit, update total profit, and record details of the sell action.
        # Update maximum hold days and period.
        # Reset positions, initial_buy, and first_buy_date
        
        if initial_buy:
            total_qty = sum([p['qty'] for p in positions])
            target_price = average_buy_price * (1 + profit_target)

            # Sell condition: Sell if the price has increased by 6.28%
            if close_price >= target_price:
                profit_inr = (close_price - average_buy_price) * total_qty
                total_profit_inr += profit_inr
                hold_days = (current_date - first_buy_date).days
                results.append({
                    'Date': current_date,
                    'Stock': ticker,
                    'Action': 'Sell',
                    'Price': close_price,
                    'RSI': rsi,
                    'Quantity': total_qty,
                    'Avg Buy Price': average_buy_price,
                    'Amount Invested': investment_per_buy * len(positions),
                    'Hold Days': hold_days,
                    'Profit INR': profit_inr,
                    'Profit%': 6.28
                })

                # Update max hold days
                if hold_days > max_hold_days:
                    max_hold_days = hold_days
                    max_hold_period = (first_buy_date, current_date)

                positions = []  # Reset positions after selling
                initial_buy = False
                first_buy_date = None  # Reset the first buy date
                continue

            
            # If the price drops to the stop-loss level and the RSI meets the criteria for additional buys (second, third, or fourth buy), calculate the quantity and update positions.
            # Record details of each additional buy.
            
            # Stop-loss conditions and second to fourth buy logic
            if len(positions) == 1 and close_price <= average_buy_price * (1 + stop_loss) and second_buy_rsi_min < rsi < second_buy_rsi_max:
                qty = calculate_quantity(investment_per_buy, close_price)
                amount_invested = qty * close_price
                positions.append({'price': close_price, 'qty': qty})
                average_buy_price = calculate_average_buy_price(positions)
                results.append({
                    'Date': current_date,
                    'Stock': ticker,
                    'Action': 'Buy 2nd',
                    'Price': close_price,
                    'RSI': rsi,
                    'Quantity': qty,
                    'Avg Buy Price': average_buy_price,
                    'Amount Invested': amount_invested,
                    'Hold Days': 0,
                    'Profit INR': 0
                })

            if len(positions) == 2 and close_price <= average_buy_price * (1 + stop_loss) and third_buy_rsi_min < rsi < third_buy_rsi_max:
                qty = calculate_quantity(investment_per_buy, close_price)
                amount_invested = qty * close_price
                positions.append({'price': close_price, 'qty': qty})
                average_buy_price = calculate_average_buy_price(positions)
                results.append({
                    'Date': current_date,
                    'Stock': ticker,
                    'Action': 'Buy 3rd',
                    'Price': close_price,
                    'RSI': rsi,
                    'Quantity': qty,
                    'Avg Buy Price': average_buy_price,
                    'Amount Invested': amount_invested,
                    'Hold Days': 0,
                    'Profit INR': 0
                })

            if len(positions) == 3 and close_price <= average_buy_price * (1 + stop_loss) and fourth_buy_rsi_min < rsi < fourth_buy_rsi_max:
                qty = calculate_quantity(investment_per_buy, close_price)
                amount_invested = qty * close_price
                positions.append({'price': close_price, 'qty': qty})
                average_buy_price = calculate_average_buy_price(positions)
                results.append({
                    'Date': current_date,
                    'Stock': ticker,
                    'Action': 'Buy 4th',
                    'Price': close_price,
                    'RSI': rsi,
                    'Quantity': qty,
                    'Avg Buy Price': average_buy_price,
                    'Amount Invested': amount_invested,
                    'Hold Days': 0,
                    'Profit INR': 0
                })

    # After iterating through all data, if positions are still open, calculate the hold days from the first buy to the end of the dataset.
    # Record details of the holding status.
    # In case the stock is still on hold
    if positions:
        hold_days = (stock_data.index[-1] - first_buy_date).days
        results.append({
            'Date': stock_data.index[-1],
            'Stock': ticker,
            'Action': 'Hold',
            'Price': stock_data.iloc[-1]['Close'],
            'RSI': stock_data.iloc[-1]['RSI'],
            'Quantity': total_qty,
            'Avg Buy Price': average_buy_price,
            'Amount Invested': investment_per_buy * len(positions),
            'Hold Days': hold_days,
            'Profit INR': 0
        })

    # Convert results list to DataFrame
    results_df = pd.DataFrame(results)
    
    # Print max hold days and period
    if max_hold_period:
        print(f"Max hold days: {max_hold_days} days, from {max_hold_period[0]} to {max_hold_period[1]}")

    # Check current status
    last_action = results_df.iloc[-1]['Action']
    if last_action == 'Hold':
        current_hold_days = results_df.iloc[-1]['Hold Days']
        print(f"Current status: Hold for {current_hold_days} days.")
    else:
        print("Currently no stock is on hold.")

    return results_df, total_profit_inr

#### Let's test this function with some example

In [21]:
# Example usage:
ticker = 'HDFCBANK.NS'
start_date = '2021-01-01'
end_date = '2024-09-01'


# Fetch stock data with RSI
stock_data = get_stock_data(ticker, start_date, end_date)

# Apply trading logic
results_df, total_profit_inr = trading_logic(stock_data)


# Print total profit earned
print(f'Total profit earned during the period: INR {total_profit_inr}')

[*********************100%***********************]  1 of 1 completed

Max hold days: 148 days, from 2024-01-17 00:00:00 to 2024-06-13 00:00:00
Currently no stock is on hold.
Total profit earned during the period: INR 18326.296752929695


#### Checking the results_df

In [22]:
results_df

,Date,Stock,Action,Price,RSI,Quantity,Avg Buy Price,Amount Invested,Hold Days,Profit INR,Profit%
0,2021-01-28,HDFCBANK.NS,Buy,1371.449951,32.322121,12,1371.449951,16457.399414,0,0.000000,NaN
1,2021-02-01,HDFCBANK.NS,Sell,1476.750000,56.501205,12,1371.449951,16000.000000,4,1263.600586,6.28
2,2021-04-12,HDFCBANK.NS,Buy,1367.050049,31.780996,12,1367.050049,16404.600586,0,0.000000,NaN
3,2021-04-28,HDFCBANK.NS,Sell,1476.800049,53.987344,12,1367.050049,16000.000000,16,1317.000000,6.28
4,2021-07-28,HDFCBANK.NS,Buy,1417.300049,34.200739,12,1417.300049,17007.600586,0,0.000000,NaN
5,2021-08-10,HDFCBANK.NS,Sell,1507.650024,60.747053,12,1417.300049,16000.000000,13,1084.199707,6.28
6,2021-11-17,HDFCBANK.NS,Buy,1530.800049,33.923324,11,1530.800049,16838.800537,0,0.000000,NaN
7,2021-12-20,HDFCBANK.NS,Buy 2nd,1425.650024,27.404192,12,1475.939167,17107.800293,0,0.000000,NaN
8,2022-04-04,HDFCBANK.NS,Sell,1656.800049,74.111632,23,1475.939167,32000.000000,138,4159.800293,6.28
9,2022-04-19,HDFCBANK.NS,Buy,1342.199951,34.858863,12,1342.199951,16106.399414,0,0.000000,NaN
